In [5]:
import sys
!{sys.executable} -m pip install --upgrade pip setuptools wheel
!{sys.executable} -m pip cache purge
!{sys.executable} -m pip install librosa



     |████████████████████████████████| 1.8 MB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 19.5 MB/s eta 0:00:01
  Using cached wheel-0.45.1-py3-none-any.whl (72 kB)
ERROR: launchpadlib 1.10.13 requires testresources, which is not installed.
ERROR: tensorflow 2.13.1 has requirement typing-extensions<4.6.0,>=3.6.6, but you'll have typing-extensions 4.13.0 which is incompatible.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.3.0
    Uninstalling setuptools-75.3.0:
      Successfully uninstalled setuptools-75.3.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Files removed: 440 (7201.0 MB)
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 48.2 MB/s eta 0:00:00
   ━━━━

In [15]:
# 🛠️ 1. Import libraries
import numpy as np
import tensorflow as tf
import os
import librosa
import random
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [32]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import os

folder_path = '/home/adityab/ADRL/A1/edge/dataset/data'
os.listdir(folder_path)  # Lists contents of the folder


['crow_dataset',
 'Rose_ringed_Parkeet',
 'Noise',
 'hen_cock-export',
 'Asean_Koel',
 '.DS_Store']

In [17]:
from scipy.io import wavfile

# Replace with the path to your .wav file
file_path = '/home/adityab/ADRL/A1/edge/dataset/data/Rose_ringed_Parkeet/Rose-ringed Parakeet.5polkukm.ingestion-54c4c64498-k9fpq.s1.wav'

# Read the WAV file
sample_rate, data = wavfile.read(file_path)

print("Sampling rate (Hz):", sample_rate)
print("Audio duration (sec):", len(data) / sample_rate)


Sampling rate (Hz): 16000
Audio duration (sec): 1.0


/tmp/ipykernel_2422261/3889777116.py:7: WavFileWarning: Reached EOF prematurely; finished at 32044 bytes, expected 32052 bytes from header.
  sample_rate, data = wavfile.read(file_path)


In [18]:
# 🛠️ 2. Prepare your dataset
DATASET_PATH = folder_path  # replace with your path
commands = ['Asean_Koel', 'crow_dataset', 'hen_cock-export', 'Noise', 'Rose_ringed_Parkeet']

X = []
Y = []

SAMPLE_RATE = 16000
DURATION = 1  # 1 second clips
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

# Load and process audio files
for label_idx, label in enumerate(commands):
    folder_path = os.path.join(DATASET_PATH, label)
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".wav"):
            file_path = os.path.join(folder_path, file_name)
            audio, sr = librosa.load(file_path, sr=SAMPLE_RATE)

            if len(audio) < SAMPLES_PER_TRACK:
                audio = np.pad(audio, (0, SAMPLES_PER_TRACK - len(audio)))
            else:
                audio = audio[:SAMPLES_PER_TRACK]

            # Simulate MFCC extraction as TensorFlow Lite Micro does
            mfcc = librosa.feature.mfcc(y=audio, sr=SAMPLE_RATE, n_mfcc=13)
            mfcc = np.expand_dims(mfcc, axis=-1)  # (13, time_steps, 1)

            X.append(mfcc)
            Y.append(label_idx)

X = np.array(X)
Y = np.array(Y)

print(f"Dataset shape: {X.shape}, Labels shape: {Y.shape}")


Dataset shape: (2518, 13, 32, 1), Labels shape: (2518,)


In [19]:
# 🛠️ 3. Train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# 🛠️ 4. Build Tiny CNN model
model = Sequential([
    Conv2D(8, (3,3), activation='relu', input_shape=X_train.shape[1:]),
    MaxPooling2D(2,2),
    Conv2D(16, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(len(commands), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 11, 30, 8)         80        
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 5, 15, 8)          0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 3, 13, 16)         1168      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 1, 6, 16)          0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 96)                0         
                                                                 
 dense_5 (Dense)             (None, 32)               

In [20]:
# 🛠️ 5. Train model
history = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_test, Y_test))

Epoch 1/100
63/63 [==============================] - 1s 8ms/step - loss: 3.9890 - accuracy: 0.5392 - val_loss: 1.2710 - val_accuracy: 0.7282
Epoch 2/100
63/63 [==============================] - 0s 5ms/step - loss: 0.7859 - accuracy: 0.7974 - val_loss: 0.6924 - val_accuracy: 0.8194
Epoch 3/100
63/63 [==============================] - 0s 5ms/step - loss: 0.4970 - accuracy: 0.8635 - val_loss: 0.5837 - val_accuracy: 0.8651
Epoch 4/100
63/63 [==============================] - 0s 5ms/step - loss: 0.3862 - accuracy: 0.8873 - val_loss: 0.4495 - val_accuracy: 0.8810
Epoch 5/100
63/63 [==============================] - 0s 5ms/step - loss: 0.2784 - accuracy: 0.9196 - val_loss: 0.3950 - val_accuracy: 0.9028
Epoch 6/100
63/63 [==============================] - 0s 5ms/step - loss: 0.2419 - accuracy: 0.9384 - val_loss: 0.3128 - val_accuracy: 0.9187
Epoch 7/100
63/63 [==============================] - 0s 5ms/step - loss: 0.2015 - accuracy: 0.9474 - val_loss: 0.3291 - val_accuracy: 0.9187
Epoch 8/100
6

In [21]:
# 🛠️ 6. Evaluate
loss, acc = model.evaluate(X_test, Y_test)
print(f"Test accuracy: {acc*100:.2f}%")

16/16 [==============================] - 0s 2ms/step - loss: 0.1799 - accuracy: 0.9643
Test accuracy: 96.43%


In [22]:
# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open("model.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmp16zakvdx/assets


INFO:tensorflow:Assets written to: /tmp/tmp16zakvdx/assets
2025-05-02 03:42:47.993757: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-05-02 03:42:47.993848: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-05-02 03:42:47.994401: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp16zakvdx
2025-05-02 03:42:47.996478: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-05-02 03:42:47.996521: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmp16zakvdx
2025-05-02 03:42:48.007781: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-05-02 03:42:48.009858: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-05-02 03:42:48.061706: I tensorflow/cc/saved_model/loader.cc:215] Running initializatio